# tsl_loto_etl 実行ノートブック（PostgreSQL / Spark / SQLite）

このノートブックは、ロト系CSVを取得→整形→特徴量化（時系列変換 + 横持ち特徴量）→ **DB（PostgreSQL）/ Parquet / SQLite** へ書き込むETLを実行します。

- 推奨の配置: `C:\tsl\pipelines\tsl_loto_etl\`（このノートも同じ場所）
- 入力SQLite: `C:\tsl\data\sqlite\loto.sqlite3`
- 出力（Parquet）: `C:\tsl\out\runs\tsl_loto_etl\tsl_loto\spark_out`
- 一時ディレクトリ（COPY用CSV）: `C:\tsl\tmp\loto_ts_pg`


## 1) セットアップ（パス・psql・表示補助）

- Windowsの `psql` 出力は文字コードが環境依存（CP932など）になりがちなので、**Decodeエラーを潰すラッパー**を使います。
- `PGPASSWORD` は `args.password` から環境変数で渡します（ノートに平文で残さない運用が理想）。


In [ ]:
import os, sys, subprocess, textwrap

# === このノートが置かれている想定フォルダ（フルパス）===
PROJECT_DIR = r"C:\tsl\pipelines\tsl_loto_etl"
os.chdir(PROJECT_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

print("cwd =", os.getcwd())
print("sys.path[0] =", sys.path[0])

# === psql.exe のフルパス（あなたの環境に合わせて）===
PSQL_BIN = r"C:\tools\pgsql\postgresql-16.11\pgsql\bin\psql.exe"
assert os.path.exists(PSQL_BIN), f"psql.exe が見つかりません: {PSQL_BIN}"

def _decode_best(b: bytes) -> str:
    if not b:
        return ""
    for enc in ("cp932", "utf-8"):
        try:
            return b.decode(enc)
        except UnicodeDecodeError:
            pass
    return b.decode("cp932", errors="replace")

def psql(sql: str, *, host: str, port: str, db: str, user: str, password: str):
    # psql結果を必ずセルに表示する。Windowsの文字コード問題を回避する。
    env = dict(os.environ)
    env["PGPASSWORD"] = password
    cmd = [
        PSQL_BIN,
        "-h", host,
        "-p", str(port),
        "-U", user,
        "-d", db,
        "-v", "ON_ERROR_STOP=1",
        "-P", "pager=off",
        "-P", "footer=on",
        "-c", sql
    ]
    r = subprocess.run(cmd, capture_output=True, env=env)
    stdout = _decode_best(r.stdout).strip()
    stderr = _decode_best(r.stderr).strip()

    print("----- SQL -----")
    print(textwrap.dedent(sql).strip())
    print("----- STDOUT -----")
    print(stdout)
    if stderr:
        print("----- STDERR -----")
        print(stderr)
    if r.returncode != 0:
        raise RuntimeError(f"psql failed (returncode={r.returncode})")

print("psql helper ready")


## 2) ETL実行（PostgreSQL例：引数を全部埋める）

- `backend='psql'` の場合、`host/port/db/user/password/schema/maintenance_db/psql_bin/tmp_dir` を埋めます。
- `psql_bin` は `psql` コマンド名でもフルパスでもOKですが、WindowsはPATH事故が多いので **フルパス推奨**です。


In [ ]:
from types import SimpleNamespace
from loto_y_ts_etl import main

args = SimpleNamespace(
    # ===== backend =====
    backend="psql",  # "psql" | "spark" | "sqlite"

    # ===== 入力（SQLite）=====
    sqlite_path=r"C:\tsl\data\sqlite\loto.sqlite3",

    # ===== 出力（Spark/Parquet）=====
    spark_out=r"C:\tsl\out\runs\tsl_loto_etl\tsl_loto\spark_out",
    spark_master=None,

    # ===== PostgreSQL（psql backendで使用）=====
    host="127.0.0.1",
    port="5432",
    db="loto_db",
    user="loto_user",
    password="CHANGE_ME",          # ← あなたのパスワードに置換
    schema="loto",                 # publicではなく専用schema推奨
    maintenance_db="postgres",     # DB自動作成で使う接続先（既にdbがあるなら実質不要）
    psql_bin=r"C:\tools\pgsql\postgresql-16.11\pgsql\bin\psql.exe",

    # ===== 出力テーブル =====
    output_table="loto_y_ts",
    hist_table="loto_hist_feat",

    # ===== 実行モード =====
    mode="replace",  # replace | append 等（実装に合わせて）
    lotos="mini,loto6,loto7,bingo5,numbers3,numbers4",

    # ===== COPY用の一時ディレクトリ（CSVが出ます）=====
    tmp_dir=r"C:\tsl\tmp\loto_ts_pg",
)

rc = main(args)
print("return_code =", rc)
assert rc == 0, "ETL が失敗しました（ログを確認してください）"


## 3) 書き込み確認（DB/スキーマ/テーブル概要・列・制約・件数・サンプル）

psqlコマンドで **現物** を確認します（GUI不要）。


In [ ]:
# 接続確認
psql(
    "SELECT current_database() AS db, current_user AS user, current_schema() AS current_schema, current_setting('search_path') AS search_path;",
    host=args.host, port=args.port, db=args.db, user=args.user, password=args.password
)

# スキーマ一覧
psql("""
SELECT nspname AS schema
FROM pg_namespace
WHERE nspname NOT LIKE 'pg_%' AND nspname <> 'information_schema'
ORDER BY 1;
""", host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

# テーブル一覧（schema内）
psql(f"""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = '{args.schema}'
ORDER BY table_name;
""", host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

# 件数
psql(f"""
SELECT
  (SELECT COUNT(*) FROM {args.schema}.{args.output_table}) AS cnt_ts,
  (SELECT COUNT(*) FROM {args.schema}.{args.hist_table})   AS cnt_hist;
""", host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

# 列定義
psql(f"""
SELECT table_name, ordinal_position, column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_schema = '{args.schema}'
  AND table_name IN ('{args.output_table}', '{args.hist_table}')
ORDER BY table_name, ordinal_position;
""", host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

# 制約
psql(f"""
SELECT
  rel.relname AS table,
  con.conname AS name,
  con.contype AS type,
  pg_get_constraintdef(con.oid) AS definition
FROM pg_constraint con
JOIN pg_class rel ON rel.oid = con.conrelid
JOIN pg_namespace n ON n.oid = rel.relnamespace
WHERE n.nspname = '{args.schema}'
  AND rel.relname IN ('{args.output_table}', '{args.hist_table}')
ORDER BY rel.relname, con.contype, con.conname;
""", host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

# インデックス
psql(f"""
SELECT schemaname, tablename, indexname, indexdef
FROM pg_indexes
WHERE schemaname = '{args.schema}'
  AND tablename IN ('{args.output_table}', '{args.hist_table}')
ORDER BY tablename, indexname;
""", host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

# サンプル
psql(f"SELECT * FROM {args.schema}.{args.output_table} ORDER BY ds DESC LIMIT 20;",
     host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)

psql(f"SELECT * FROM {args.schema}.{args.hist_table} ORDER BY ds DESC LIMIT 20;",
     host=args.host, port=args.port, db=args.db, user=args.user, password=args.password)


## 4) SQLiteで確認したい場合（オプション）

DBを使わずローカルだけで確認したいときの補助です。


In [ ]:
import sqlite3
import pandas as pd

sqlite_path = r"C:\tsl\data\sqlite\loto.sqlite3"
conn = sqlite3.connect(sqlite_path)

df_tables = pd.read_sql_query(
    "SELECT name AS table_name FROM sqlite_master WHERE type='table' ORDER BY name;",
    conn
)
display(df_tables)

def sqlite_table_info(table: str) -> pd.DataFrame:
    return pd.read_sql_query(f"PRAGMA table_info('{table}');", conn)

display(sqlite_table_info("loto_y_ts"))
display(sqlite_table_info("loto_hist_feat"))

df_counts = pd.DataFrame({
    "table": ["loto_y_ts","loto_hist_feat"],
    "rows": [
        pd.read_sql_query("SELECT COUNT(*) AS n FROM loto_y_ts;", conn)["n"][0],
        pd.read_sql_query("SELECT COUNT(*) AS n FROM loto_hist_feat;", conn)["n"][0],
    ]
})
display(df_counts)

display(pd.read_sql_query("SELECT * FROM loto_y_ts LIMIT 20;", conn))
display(pd.read_sql_query("SELECT * FROM loto_hist_feat LIMIT 20;", conn))
